<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
print(yelp.shape)

yelp.head()

(10000, 9)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
# using spacy and the tokenizer
# maybe will have to switch to md or sm if it causes my comp problems
import spacy
from spacy.tokenizer import Tokenizer

In [4]:
# load the model
nlp = spacy.load('en_core_web_lg')

In [5]:
# instantiate the tokenizer 
tokenizer = Tokenizer(nlp.vocab)

In [6]:
# f(x) to tokenize a single doc
def tokenize(doc):
    return [token.text for token in tokenizer(doc)]

In [7]:
# Column of tokenized text
yelp['tokens'] = [tokenize(doc) for doc in yelp['text'].values]

In [8]:
# and it seems to have worked
yelp['tokens'].head(10)

0    [BEWARE!!!, FAKE,, FAKE,, FAKE....We, also, ow...
1    [Came, here, for, lunch, Togo., Service, was, ...
2    [I've, been, to, Vegas, dozens, of, times, and...
3    [We, went, here, on, a, night, where, they, cl...
4    [3.5, to, 4, stars, \n\n, Not, bad, for, the, ...
5    [Tasty,, fast, casual, Latin, street, food.,  ...
6    [This, show, is, absolutely, amazing!!, What, ...
7    [Came, for, the, Pho, and, really, enjoyed, it...
8    [Absolutely, the, most, Unique, experience, in...
9    [Wow., I, walked, in, and, sat, at, the, bar, ...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [4]:
# join back the token lists into strings
data =  [' '.join(s) for s in yelp['tokens']]

len(data)

KeyError: 'tokens'

In [5]:
# using the standard english stop words, and cutting out 5% outliers
vect = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.95)

# fit to the data
vect.fit(data)

# vectorize data into sparse matrix
sparse = vect.transform(data)

# create a df of the matrix but filled in with zeros
dtm = pd.DataFrame(sparse.todense(), columns=vect.get_feature_names())

dtm.head()

NameError: name 'data' is not defined

In [12]:
# instantiate the model very simply
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

# fit to dataframe from vectorized data
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [13]:
# fake review to test model
fake_review = ['''
    This product is terrible. It broke the first time I used it.
    I want my money back
''']

In [14]:
# vectorize review
search = vect.transform(fake_review)

# run the search
nn.kneighbors(search.todense())

(array([[0.        , 0.60053305, 0.60053305, 0.60053305, 0.82027879]]),
 array([[9037, 9181, 5547, 8411, 5518]]))

### Model not great but these reviews all do have strongly negative sentiments so it nailed that.

In [15]:
data[9037]

'If you want to be seen by a DR. who looks at his laptop more than he looks at you or spend time with you or even listen to you, this is the one to go to!'

In [16]:
data[8411]

"The man that answers the phone has a horrible attitude. If they do NOT want to do Delivary shouldn't be advertising as so."

In [17]:
data[5547]

'Over priced hipster jive ass baking. The chocolate babka is getting them the extra star but you can pass on the rest unless you want some greasy croissants or some other stomach ache inducing nonsense. Keep on walking. Nothing to see here.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD

In [7]:
# Instantiate the steps of the pipeline
svd = TruncatedSVD(n_iter=7)
vect = TfidfVectorizer(tokenizer=tokenize)
clf = RandomForestClassifier()

# Create the pipelines
lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

NameError: name 'tokenize' is not defined

In [20]:
# set parameters to tune
parameters = {
    'lsi__vect__max_df': [0.925, 0.95, 1.0],
    'lsi__vect__ngram_range': [(1,1), (1,2)],
    'lsi__svd__n_components': [2,5,10],
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [3,4,5]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# create X and y
X = yelp['text']
y = yelp['stars']

# couldnt run this - my comp was freezing up
grid_search = GridSearchCV(pipe, parameters, cv=3, n_iter=5, n_jobs=4, verbose=10)
grid_search.fit(X, y)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [8]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

In [9]:
# retokenize using gensim and taking out stopwords
data = []

for val in yelp['text']:
    doc_tokens = [token for token in simple_preprocess(val) if token not in STOPWORDS]
    data.append(doc_tokens)
    
yelp['tokens'] = data

yelp['tokens'].head(10)

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
5    [tasty, fast, casual, latin, street, food, men...
6    [absolutely, amazing, incredible, production, ...
7    [came, pho, enjoyed, got, pm, busy, got, serve...
8    [absolutely, unique, experience, nail, shop, f...
9    [wow, walked, sat, bar, minutes, bartenders, w...
Name: tokens, dtype: object

Learn the vocubalary of the yelp data:

In [10]:
id2word = Dictionary(yelp['tokens'])

Create a bag of words representation of the entire corpus

In [11]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [12]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

In [38]:
lda.print_topics()

[(0,
  '0.010*"place" + 0.010*"food" + 0.009*"time" + 0.008*"like" + 0.007*"great" + 0.007*"good" + 0.006*"service" + 0.005*"best" + 0.004*"got" + 0.004*"ve"'),
 (1,
  '0.011*"place" + 0.010*"good" + 0.010*"food" + 0.009*"great" + 0.008*"like" + 0.007*"time" + 0.005*"service" + 0.004*"best" + 0.004*"ve" + 0.004*"restaurant"'),
 (2,
  '0.011*"place" + 0.011*"good" + 0.010*"food" + 0.009*"service" + 0.008*"great" + 0.007*"time" + 0.007*"like" + 0.005*"ve" + 0.004*"got" + 0.004*"best"'),
 (3,
  '0.009*"great" + 0.008*"food" + 0.008*"good" + 0.007*"time" + 0.007*"service" + 0.007*"like" + 0.007*"place" + 0.004*"nice" + 0.004*"got" + 0.004*"staff"'),
 (4,
  '0.013*"food" + 0.012*"great" + 0.010*"good" + 0.009*"place" + 0.006*"time" + 0.006*"like" + 0.005*"service" + 0.005*"best" + 0.004*"come" + 0.004*"nice"'),
 (5,
  '0.012*"place" + 0.010*"food" + 0.010*"great" + 0.010*"good" + 0.009*"service" + 0.008*"time" + 0.007*"like" + 0.005*"got" + 0.004*"love" + 0.004*"ordered"'),
 (6,
  '0.011*"f

In [13]:
import re

words = [re.findall('"([^"]*)"', t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:3]) for t in words]

for id, t in enumerate(topics):
    print(f'----- Topic {id} -----')
    print(t, end='\n\n')

----- Topic 0 -----
food good great

----- Topic 1 -----
food good like

----- Topic 2 -----
service time place

----- Topic 3 -----
food like good

----- Topic 4 -----
place food great

----- Topic 5 -----
place good food

----- Topic 6 -----
good food time

----- Topic 7 -----
food great place

----- Topic 8 -----
place good food

----- Topic 9 -----
great good food



Create 1-2 visualizations of the results

In [20]:
import gensim
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

KeyboardInterrupt: 

pyLDAvis was spinning for a while, I wasnt sure if it was breaking my computer. 

1. Initially ran Topic Modelling with 10 topics. This produced many topics that were saying essentially the same thing.

2. A common them is people liking the service and the food.

In [46]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics =5 # You can change this parameter
                  )

/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


In [47]:
words = [re.findall('"([^"]*)"', t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:3]) for t in words]

for id, t in enumerate(topics):
    print(f'----- Topic {id} -----')
    print(t, end='\n\n')

----- Topic 0 -----
great food place

----- Topic 1 -----
good food place

----- Topic 2 -----
place food time

----- Topic 3 -----
good food great

----- Topic 4 -----
food place good



 - These topics really aren't amazing. Seem to be looking at alot of positive reviews.
 
 - Going to add add some words to STOPWORDS and keep more words as the topic

In [59]:
STOPWORDS = set(STOPWORDS).union(set(['good', 'great', 'place']))

# retokenize using gensim and taking out stopwords
data = []

for val in yelp['text']:
    doc_tokens = [token for token in simple_preprocess(val) if token not in STOPWORDS]
    data.append(doc_tokens)
    
yelp['tokens'] = data

yelp['tokens'].head(10)

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
5    [tasty, fast, casual, latin, street, food, men...
6    [absolutely, amazing, incredible, production, ...
7    [came, pho, enjoyed, got, pm, busy, got, serve...
8    [absolutely, unique, experience, nail, shop, f...
9    [wow, walked, sat, bar, minutes, bartenders, w...
Name: tokens, dtype: object

In [60]:
id2word = Dictionary(yelp['tokens'])
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 5 # You can change this parameter
                  )

/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/jay/anaconda3/envs/nlp-u4-s1/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


In [61]:
words = [re.findall('"([^"]*)"', t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:5]) for t in words]

for id, t in enumerate(topics):
    print(f'----- Topic {id} -----')
    print(t, end='\n\n')

----- Topic 0 -----
food like service time got

----- Topic 1 -----
food service time like ve

----- Topic 2 -----
service food time like got

----- Topic 3 -----
food like time service got

----- Topic 4 -----
food like time service got



Playing with the different words in the STOPWORDS, I kept getting similar results in that people are talking about the same kinds of things. These reviews all seem to be about restaurants because everyone is talking about (food, service, time). 
I was a bit unsure if to remove a whole bunch of words from the top but that seemed a bit much. I felt it was ok to remove (good, great) because we can separate by ratings and we'll know if people are talking positive or negative.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)